In [1]:
import numpy as np

In [2]:
train_images=np.load('data/train-images.npy')

In [3]:
train_images=train_images.reshape((-1,28*28))

In [4]:
train_labels=np.load('data/train-labels.npy')

In [5]:
def sigmoid(x):
    return 1/(np.exp(-x)+1)

In [6]:
def d_sigmoid(x):
    return (np.exp(-x))/((np.exp(-x)+1)**2)

In [7]:
def softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)

In [8]:
def d_softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)*(1-exp_element/np.sum(exp_element,axis=0))

In [9]:
def init(x,y):
    return np.random.uniform(-1.,1.,(x,y))/np.sqrt(x*y)

In [10]:
np.random.seed(17)

In [11]:
l1=init(28*28,128)

In [12]:
l2=init(128,10)

In [13]:
import apple

In [14]:
ssoftmax=apple.jit('''
λxs.
  { m ⟜ (⋉)/* _1 xs; a ⟜ [e:(x-m)]`{0} xs
  ; sum ← [(+)/x]
  ; n ⟜ sum`{1} (a::M float)
  ; |:(([(%x)'y]`{0,1} n a))
  }
''')

In [15]:
vsigmoid=apple.jit("([1%(1+ℯ(_x))]`{0})")

In [16]:
vmap=apple.jit("((λn.[?x=n,.1::float,.0]'irange 0 9 1)')")

In [17]:
mmul=apple.jit("[x%.(y::M float)]")

In [18]:
def fw_bw(x,y):
    # targets = np.zeros((len(y),10), np.float64)
    # targets[range(targets.shape[0]),y] = 1
    targets=apple.f(vmap,y)

    # x_l1p=x.dot(l1)
    x_l1p=apple.f(mmul,x,l1)
    # x_sigmoid=sigmoid(x_l1p)
    x_sigmoid=apple.f(vsigmoid,x_l1p)
    # x_l2p=x_sigmoid.dot(l2)
    x_l2p=apple.f(mmul,x_sigmoid,l2)
    # out=softmax(x_l2p)
    out=apple.f(ssoftmax,x_l2p)

    error=2*(out-targets)/out.shape[0]*d_softmax(x_l2p)
    update_l2=x_sigmoid.T@error


    error=((l2).dot(error.T)).T*d_sigmoid(x_l1p)
    update_l1=x.T@error

    return out,update_l1,update_l2

In [19]:
fw_bw(train_images,train_labels)

(array([[1.66786736e-05, 1.66682799e-05, 1.66700078e-05, ...,
         1.66680789e-05, 1.66606107e-05, 1.66722308e-05],
        [1.66795898e-05, 1.66693587e-05, 1.66627420e-05, ...,
         1.66583628e-05, 1.66524988e-05, 1.66816094e-05],
        [1.66451471e-05, 1.66741052e-05, 1.66730918e-05, ...,
         1.66759087e-05, 1.66643422e-05, 1.66748893e-05],
        ...,
        [1.66659155e-05, 1.66637678e-05, 1.66669794e-05, ...,
         1.66895096e-05, 1.66579513e-05, 1.66598272e-05],
        [1.66731368e-05, 1.66652597e-05, 1.66789388e-05, ...,
         1.66591687e-05, 1.66705506e-05, 1.66714626e-05],
        [1.66676569e-05, 1.66739332e-05, 1.66676227e-05, ...,
         1.66491424e-05, 1.66624448e-05, 1.66490809e-05]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[-1.6381